## Set up

### Import modules

In [13]:
import os
from pathlib import Path
#import re
from datetime import date,datetime
from configparser import ConfigParser
import psycopg2
#from psycopg2.extensions import AsIs
import pandas as pd

### Working dir and relative paths

In [2]:
import pyprojroot
repodir = pyprojroot.find_root(pyprojroot.has_dir(".git"))

### Database connection

In [4]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'psqlawseco'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

## Database queries

In [7]:
conn = psycopg2.connect(**db)
cur = conn.cursor()

In [8]:
qry = """
SELECT version, language, COUNT(distinct code) as n,
    MIN(DATE(update)) as date1, MAX(DATE(update)) as date2 
FROM efg_distribution 
GROUP BY version, language 
ORDER BY date1;
"""

In [9]:
cur.execute(qry )
versions = cur.fetchall()

In [11]:
cur.close()
conn.commit()
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


## View data

In [14]:
short_table_df = pd.DataFrame(versions,
                           columns=("version","language", "n", "date1", "date2"))

In [15]:
short_table_df

,version,language,n,date1,date2
0,v1.0,en,104,2020-01-20,2020-05-31
1,v1.01,en,1,2020-02-21,2020-02-21
2,v1.2,en,1,2020-05-14,2020-05-14
3,v2.0,en,108,2020-05-24,2020-09-04
4,v2.01,en,108,2021-12-31,2021-12-31
5,v2.1,en,110,2022-04-06,2022-06-29
6,v2.1,es,110,2024-12-18,2025-02-19
